 **Startups in 2021 end**

In [ ]:
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime as dt
import numpy as np
import pandas as pd


In [ ]:
# загрузим данные по годам и исследуем их

startup = pd.read_csv('/content/Startups in 2021 end.csv', sep=',', index_col=0)
startup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 936 entries, 0 to 935
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Company           936 non-null    object
 1   Valuation ($B)    936 non-null    object
 2   Date Joined       936 non-null    object
 3   Country           936 non-null    object
 4   City              921 non-null    object
 5   Industry          936 non-null    object
 6   Select Investors  935 non-null    object
dtypes: object(7)
memory usage: 58.5+ KB


In [ ]:
# поправим запятые в названиях и пропуски в городах
startup['City'].loc[startup['City'].isna()] = startup['Country']
startup['Country'].loc[startup['Country'] == 'Hong Kong'] = startup['Country'].loc[startup['Country'] == 'Hong Kong'].str.replace('Hong Kong', 'China')
startup['Country'] = startup['Country'].str.replace(',', '', regex=True)
startup['Industry'] = startup['Industry'].str.replace(r'Finttech', 'Fintech', regex=True)
startup['Industry'] = startup['Industry'].str.title()
startup.groupby('Country').count().tail()

,Company,Valuation ($B),Date Joined,City,Industry,Select Investors
Country,,,,,,
Turkey,2,2,2,2,2,2
United Arab Emirates,3,3,3,3,3,3
United Kingdom,37,37,37,37,37,37
United States,478,478,478,478,478,478
Vietnam,1,1,1,1,1,1


In [ ]:
# поправим значения дат и оценочной стоимости
startup['Date Joined'] = pd.to_datetime(startup['Date Joined'], format='%m/%d/%Y')
startup['Year joined'] = startup['Date Joined'].dt.to_period('Y').astype(str).astype(int)
startup['Valuation ($B)'] = startup['Valuation ($B)'].str.replace(r'$', '', regex=True).astype(float)
startup.head(5)

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Investors,Year joined
0,Bytedance,140.0,2017-04-07,China,Beijing,Artificial Intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2017
1,SpaceX,100.3,2012-12-01,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2012
2,Stripe,95.0,2014-01-23,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2014
3,Klarna,45.6,2011-12-12,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2011
4,Canva,40.0,2018-01-08,Australia,Surry Hills,Internet Software & Services,"Sequoia Capital China, Blackbird Ventures, Mat...",2018


In [ ]:
# посмотрим количество компаний по годам
years = startup.resample('Y', on='Date Joined')['Company'].size()
fig = px.line(years, title='Count startups by the year')
fig.show()

С 2014 года наблюдаем рост количества стартапов. Это позволяет нам предположить, что в 2014 году произошло что-то, что простимулировало данное направление деятельности.

In [ ]:
# посмотрим распределение по странам
country = startup.copy()
country = country.groupby('Country').size().sort_values(ascending=False).head(10)
fig = px.bar(country,
             title='TOP-10 country by q-ty startups',)
fig.show()

На первом месте по количеству стартапов Unated States

In [ ]:
# построим количество по странам с разбивкой на города
df = startup.copy()
df = pd.DataFrame(startup.groupby(['Country', 'City']).size().sort_values(ascending=False).head(10))
df = df.reset_index()
df.columns = ['Country', 'City', 'Value']
fig = px.treemap(df, path=[px.Constant('world'),'Country', 'City'],
                 values='Value',
                 color_continuous_scale='deep')
fig.show()

Больше всего стартапов с 2014 по 2021 год было зарегистрировано в Unated States, город San Francisco

In [ ]:
# посмотрим инвестиции в разрезе отраслей по годам
ind = startup.groupby(['Industry', 'Year joined']).sum('Valuation ($B)')
ind.reset_index(inplace=True)
fig = px.scatter(ind,
                 x='Year joined',
                 y='Industry',
                 size='Valuation ($B)',
                 title='Yearly Valuation ($B) by Industry')
fig.show()

Максимальные инвестиции за всё время были в 2021 году в Fintech - 295.4 $B

In [ ]:
# посмотрим на активность инвесторов в разрезе стран
inv = startup.copy()
inv = inv[['Select Investors',  'Country']]
investors = inv['Select Investors'].str.split(', ', expand=True)
rez = pd.concat([inv,investors], axis=1 )
rez.drop('Select Investors', axis=1, inplace=True)

rez = pd.melt(rez, 'Country').dropna()

# сразу поправим записи
rez['value'] = rez['value'].str.title()

# возмем пятерку самых активных инвесторов
act_inv = rez.copy()
act_inv = act_inv.groupby('value').agg('count').sort_values('variable', ascending=False).head(5).reset_index()
act_inv

,value,Country,variable
0,Accel,50,50
1,Sequoia Capital China,47,47
2,Tiger Global Management,44,44
3,Sequoia Capital,40,40
4,Andreessen Horowitz,39,39


In [ ]:
# посмотрим, в каких странах они работали
rez = rez.groupby(['value', 'Country']).sum().sort_values(['value', 'Country', 'variable'], ascending =False)

rez = rez.reset_index()
rez = rez[rez['variable'] > 0]
rez = rez[rez['value'].isin(act_inv['value'])].groupby(['value', 'Country']).sum().sort_values('value', ascending =False).head(100)
#
rez = rez.reset_index()
rez.columns = ['Company', 'Country', 'Investment q-ty']

In [ ]:
fig = px.bar(rez, x='Company', y='Investment q-ty',
             color='Country',
             title='TOP-5 investors',
             text_auto=True)
fig.show()

Размера инвестиций по компаниям мы посчитать не сможем, потому что у нас общая сумма по всем инвесторам. Однако и на этой визуалицации мы видим, что "акулы" инвестиций предпочитают Unated States